In [1]:
import os
import pathlib
import json

In [2]:
import pandas as pd 
import numpy as np

### enter bulletin directory 

In [3]:
bulletin_dir = 'AMJ_2019'

In [4]:
dpath = pathlib.Path.cwd().parents[1] / 'bulletin' / bulletin_dir

In [5]:
dpath

PosixPath('/home/nicolasf/operational/ICU/ops/bulletin/AMJ_2019')

### reads the country dictionnary for the TRMM rainfall time-series

In [6]:
with open(pathlib.Path.cwd().parents[1] / 'resources' / 'dict_countries.json', 'r') as fj: 
    dict_countries = json.load(fj)

### read the forecast table 

In [7]:
forecast = pd.read_csv(dpath / 'copernicus_forecasts_{}.csv'.format(bulletin_dir), index_col=0)

In [8]:
forecast.head()

,T1,T2,T3,country_fname,std,CONFIDENCE
country_name,,,,,,
New Caledonia,72,17,11,New_Caledonia,10.979141,High
Pitcairn Islands,26,27,47,Pitcairn_Islands,16.047438,Moderate-High
Papua New Guinea,43,31,26,Papua_New_Guinea,13.516244,High
Solomon Islands,26,32,42,Solomon_Islands,19.170869,Moderate-High
Kiribati: Phoenix Islands,1,2,97,Kiribati:_Phoenix_Islands,16.478053,Moderate-High


In [9]:
def terciles_to_category(df, colnames=['T1','T2','T3'], mindiff=10, climdiff=5): 
    """
    convert a tercile probabilistic forecast into a categorical forecast
    
    if the difference between the maximum probability and the next one is >= mindiff 
    (10 is default) then it's either 'ABOVE', 'NEAR NORMAL' or 'BELOW' depending on the max 
    probability 
    
    if the difference between consecutive probabilities is less than mindiff (10)
    but greater than climdiff (5) then we end up with either 'AVG - BELOW' or 'AVG - ABOVE'
    
    if the difference between consecutive probabilities is less than 5%, it's a CLIMATOLOGY 
    forecasts 
    """
    
    T1, T2, T3 = colnames
    
    cats = []
    for i, row in df.iterrows(): 
        fcst = row.loc[colnames,].astype('int')
        if (fcst.idxmax()  == T3) & ((fcst.loc[T3] - fcst.loc[T2]) >= mindiff): 
            cat = 'ABOVE'
        elif (fcst.idxmax()  == T3) & ((fcst.loc[T3] - fcst.loc[T2]) < mindiff) & ((fcst.loc[T3] - fcst.loc[T2]) > climdiff):
            cat = 'AVG - ABOVE'
        elif (fcst.idxmax()  == T1) & ((fcst.loc[T1] - fcst.loc[T2]) >= mindiff):
            cat = 'BELOW'
        elif (fcst.idxmax()  == T1) & ((fcst.loc[T1] - fcst.loc[T2]) < mindiff) & ((fcst.loc[T1] - fcst.loc[T2]) > climdiff):
            cat = 'AVG - BELOW'
        elif (fcst.idxmax()  == T2) & ((fcst.loc[T2] - fcst.loc[T1]) >= mindiff) & ((fcst.loc[T2] - fcst.loc[T3]) >= mindiff): 
            cat = 'NEAR NORMAL'
        else: 
            cat = 'CLIMATOLOGY'
        cats.append(cat)
    return np.array(cats)

In [10]:
forecast.loc[:,'CATEGORY'] = terciles_to_category(forecast)

In [11]:
forecast

,T1,T2,T3,country_fname,std,CONFIDENCE,CATEGORY
country_name,,,,,,,
New Caledonia,72,17,11,New_Caledonia,10.979141,High,BELOW
Pitcairn Islands,26,27,47,Pitcairn_Islands,16.047438,Moderate-High,ABOVE
Papua New Guinea,43,31,26,Papua_New_Guinea,13.516244,High,BELOW
Solomon Islands,26,32,42,Solomon_Islands,19.170869,Moderate-High,ABOVE
Kiribati: Phoenix Islands,1,2,97,Kiribati:_Phoenix_Islands,16.478053,Moderate-High,ABOVE
Fiji,39,32,29,Fiji,16.142961,Moderate-High,AVG - BELOW
Tuvalu,15,18,67,Tuvalu,17.881378,Moderate-High,ABOVE
Northern Cook Islands,38,31,31,Northern_Cook_Islands,15.491184,Moderate-High,AVG - BELOW
Kiribati: Line Islands,2,8,90,Kiribati:_Line_Islands,15.530979,Moderate-High,ABOVE


### saves to the folder for inclusion in the PPT 

#### This is the table that appears on page 2 of the ICu bulletin

In [12]:
ppt_forecast_table = forecast.loc[:,['T1','T2','T3','CATEGORY','CONFIDENCE']]

In [13]:
ppt_forecast_table = ppt_forecast_table.sort_values(by=['T3','T2','T1'], ascending=False)

In [14]:
ppt_forecast_table = ppt_forecast_table.rename(index={'Tuamotu / Gambier Islands':'Tuamotu Islands'})

In [15]:
ppt_forecast_table = ppt_forecast_table.rename(index={'Federated States of Micronesia':'FSM'})

In [16]:
ppt_forecast_table.to_csv(dpath / 'ICU_forecast_table_for_PPT_{}.csv'.format(bulletin_dir))

### code the categories for the water stress calculations

In [17]:
dict_replace = {}

dict_replace['ABOVE'] = 4

dict_replace['AVG - ABOVE'] = 2
dict_replace['AVG-ABOVE'] = 2
dict_replace['AVG – ABOVE'] = 2
dict_replace['AVG–ABOVE'] = 2

dict_replace['NEAR NORMAL'] = 0
dict_replace['NEAR NORMAL '] = 0
dict_replace['CLIMATOLOGY'] = 0


dict_replace['AVG - BELOW'] = -3
dict_replace['AVG-BELOW'] = -3
dict_replace['AVG – BELOW'] = -3
dict_replace['AVG–BELOW'] = -3

dict_replace['BELOW'] = -5

dict_replace['CLIMATOLOGY'] = -666

In [18]:
forecast = forecast.replace({'CATEGORY':dict_replace})

In [19]:
forecast

,T1,T2,T3,country_fname,std,CONFIDENCE,CATEGORY
country_name,,,,,,,
New Caledonia,72,17,11,New_Caledonia,10.979141,High,-5
Pitcairn Islands,26,27,47,Pitcairn_Islands,16.047438,Moderate-High,4
Papua New Guinea,43,31,26,Papua_New_Guinea,13.516244,High,-5
Solomon Islands,26,32,42,Solomon_Islands,19.170869,Moderate-High,4
Kiribati: Phoenix Islands,1,2,97,Kiribati:_Phoenix_Islands,16.478053,Moderate-High,4
Fiji,39,32,29,Fiji,16.142961,Moderate-High,-3
Tuvalu,15,18,67,Tuvalu,17.881378,Moderate-High,4
Northern Cook Islands,38,31,31,Northern_Cook_Islands,15.491184,Moderate-High,-3
Kiribati: Line Islands,2,8,90,Kiribati:_Line_Islands,15.530979,Moderate-High,4


### changes the names of the columns etc and saves the forecast table for mapping 

In [20]:
table_for_mapping = forecast.copy()

In [21]:
table_for_mapping.loc[:,'COUNTRYNAM'] = table_for_mapping.index

In [22]:
table_for_mapping.rename({'CATEGORY':'Stress'}, axis=1, inplace=True)

In [23]:
table_for_mapping = table_for_mapping.loc[:,['Stress','COUNTRYNAM']]

In [24]:
table_for_mapping.index.name = None

In [25]:
table_for_mapping.to_csv(dpath / 'ICU_forecast_table_for_mapping.csv')

### Now read the last 6 months rainfall categories from TRMM

In [26]:
last_6_months = pd.read_csv(dpath / 'last_6_months_TRMM_categories.csv', index_col=0)

In [27]:
last_6_months.set_index('month', inplace=True)

In [28]:
last_6_months

,Palau,Papua_New_Guinea,Society_Islands,Niue,Kiribati_Phoenix_Islands,Wallis_and_Futuna,Marquesas,Fiji,Kiribati_Line_Islands,Vanuatu_North,...,Tuvalu,Solomon_Islands,Tonga,Pitcairn_Islands,Guam,Northern_Marianas,Federated_States_of_Micronesia,Marshall_Islands,American_Samoa,year
month,,,,,,,,,,,,,,,,,,,,,
10,Normal,Below,Well below,Below,Above,Well above,Below,Well above,Below,Below,...,Below,Below,Above,Well above,Below,Normal,Well below,Normal,Well above,2018
11,Normal,Normal,Below,Well below,Well above,Below,Normal,Normal,Above,Well below,...,Well above,Well above,Well below,Below,Above,Above,Below,Below,Below,2018
12,Normal,Above,Well below,Well above,Normal,Well above,Above,Well above,Normal,Well above,...,Normal,Well above,Well above,Below,Above,Below,Below,Above,Well above,2018
1,Below,Normal,Above,Above,Normal,Above,Normal,Well above,Above,Below,...,Below,Below,Well above,Well below,Normal,Below,Well above,Normal,Well below,2019
2,Well below,Well below,Normal,Below,Well above,Well above,Well below,Normal,Above,Above,...,Well above,Above,Well above,Above,Well above,Above,Below,Above,Well above,2019
3,Above,Well above,Above,Well below,Above,Normal,Normal,Below,Well above,Well below,...,Well below,Well below,Above,Well above,Below,Well below,Above,Normal,Below,2019


In [29]:
last_6_months = last_6_months.T

In [30]:
dict_replace = {}
dict_replace['Well below'] = -5 
dict_replace['Below'] = -3 
dict_replace['Normal'] = 0
dict_replace['Above'] = 2
dict_replace['Well above'] = 4

In [31]:
last_6_months.head()

month,10,11,12,1,2,3
Palau,Normal,Normal,Normal,Below,Well below,Above
Papua_New_Guinea,Below,Normal,Above,Normal,Well below,Well above
Society_Islands,Well below,Below,Well below,Above,Normal,Above
Niue,Below,Well below,Well above,Above,Below,Well below
Kiribati_Phoenix_Islands,Above,Well above,Normal,Normal,Well above,Above


In [32]:
last_6_months  = last_6_months.replace(dict_replace)

In [33]:
last_6_months = last_6_months.sort_index()

In [34]:
last_6_months.columns = range(-6,0)

In [35]:
last_6_months = last_6_months.rename(index=dict_countries)

In [36]:
last_6_months.head()

,-6,-5,-4,-3,-2,-1
American Samoa,4,-3,4,-5,4,-3
Austral Islands,2,-3,0,4,4,-3
Federated States of Micronesia,-5,-3,-3,4,-3,2
Fiji,4,0,4,4,0,-3
Guam,-3,2,2,0,4,-3


In [37]:
last_6_months.index.name = 'country_name'

### now calculates the water stress itself 

In [38]:
water_stress = forecast.loc[:,['CATEGORY']]

In [39]:
water_stress = water_stress.merge(last_6_months, left_index=True, right_index=True)

### replace the code for climatology (-666) with 0 (i.e. equivalent to normal) for calculation of the water stress level

In [40]:
water_stress = water_stress.replace({'CATEGORY':{-666:0}})

In [41]:
water_stress.loc[:,'Stress'] = water_stress.loc[:,-1] +  water_stress.loc[:,-2] + \
0.8 * water_stress.loc[:,-3] + \
0.6 * water_stress.loc[:,-4] + \
0.4 * water_stress.loc[:,-5] + \
0.2 * water_stress.loc[:,-6] + \
water_stress.loc[:,'CATEGORY']

In [42]:
water_stress.head()

,CATEGORY,-6,-5,-4,-3,-2,-1,Stress
country_name,,,,,,,,
New Caledonia,-5,2,2,4,-5,2,-5,-8.4
Pitcairn Islands,4,4,-3,-3,-5,2,4,3.8
Papua New Guinea,-5,-3,0,2,0,-5,4,-5.4
Solomon Islands,4,-3,4,4,-3,2,-5,2.0
Kiribati: Phoenix Islands,4,2,4,0,0,4,2,12.0


### now determine the water stress catergory from the water stress value

In [43]:
bins = [-np.inf, -10, -7, -2, 2, 6, 11, np.inf]

In [44]:
labels = [-3, -2, -1, 0, 1, 2, 3]

In [45]:
water_stress.loc[:,'stress_category'] = pd.cut(water_stress.Stress.values.flatten(), bins=bins, labels=labels)

In [46]:
water_stress

,CATEGORY,-6,-5,-4,-3,-2,-1,Stress,stress_category
country_name,,,,,,,,,
New Caledonia,-5,2,2,4,-5,2,-5,-8.4,-2
Pitcairn Islands,4,4,-3,-3,-5,2,4,3.8,1
Papua New Guinea,-5,-3,0,2,0,-5,4,-5.4,-1
Solomon Islands,4,-3,4,4,-3,2,-5,2.0,0
Kiribati: Phoenix Islands,4,2,4,0,0,4,2,12.0,3
Fiji,-3,4,0,4,4,0,-3,0.4,0
Tuvalu,4,-3,4,0,-3,4,-5,1.6,0
Northern Cook Islands,-3,0,-3,-3,2,2,-5,-7.4,-2
Kiribati: Line Islands,4,-3,2,0,2,2,4,11.8,3


In [47]:
water_stress.index.name = 'locationID'

In [48]:
dpath

PosixPath('/home/nicolasf/operational/ICU/ops/bulletin/AMJ_2019')

### now saves for mapping 

In [49]:
water_stress.to_csv(dpath / 'ICU_stress_table_for_mapping.csv')